# Block Tokenizer and Vector Embedding (block2vec)

Tokenizes blocks using BPE and gives tokens vector embedding using word2vec approach

In [ ]:
from blockgen.tokenizers.block_tokenizer import check_vocab

vocab = check_vocab(["data/raw/1.schematic"])
with open("vocab.txt", "w") as f:
    for item in vocab:
        f.write(f"{item}\n")

ModuleNotFoundError: No module named 'src'